In [1]:
import pandas as pd
import nltk
import time

from datetime import datetime 
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup

In [2]:
pages = 10
news_list = []
target_url = 'https://www.fxstreet.com/news?q=&hPP=17&idx=FxsIndexPro&p='
output_csv = 'fxstreet.csv'

In [3]:
for page_no in range(0, pages) :
    url = target_url + str(page_no) 
    
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)
    time.sleep(5)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    
    div = soup.find('div', class_= "ais-hits")
    
    for news_div in soup.find_all('div', class_= 'ais-hits--item'):
        h4 = news_div.find('h4')
        news_url = h4.a.get('href')
        req = Request(news_url , headers={'User-Agent': 'Mozilla/5.0'})
        article_page = urlopen(req).read()
        article_soup = BeautifulSoup(article_page, "html.parser")

        news_title_a = article_soup.find("h1", class_ = "fxs_headline_large").text
        news_body = ""
        for p in article_soup.find("div", id = "fxs_article_content").find_all("p"):
            news_body += p.text
        
        news_date = article_soup.find('time').text
        news_date = news_date.split(" ")[0]
        news_date = datetime.strptime(news_date, "%m/%d/%Y").date()
        news_href = h4.a.get('href')
        news_list.append([page_no, news_date, news_title_a, news_body ])

    news_dataset = pd.DataFrame(news_list, columns = ['page', 'date','title','body'] )
    news_dataset["source"] = "fxstreet"

news_dataset.to_csv(output_csv, index= False)